In [104]:
import pandas as pd
import glob
from datetime import datetime, timedelta, date
import preprocessor as prep
import string
import re
import nltk
pd.set_option('display.max_colwidth', -1)

Loop over the csv files per company 

New York Stock exchange opening hours are (UTC) 14:30 - 21:00

In [47]:
def get_tweets(file, evening):
    data = pd.read_csv(file)
    data['Created at'] = pd.to_datetime(data['Created at'])
    if evening:
        closure_time = datetime.combine(data['Created at'].max().date(), datetime.strptime('2100','%H%M').time())
        data = data[data['Created at'] >= closure_time]
    else:
        open_time = datetime.combine(data['Created at'].max().date(), datetime.strptime('1430','%H%M').time())
        data = data[data['Created at'] <= open_time]
    return data[['Created at', 'Content']]

In [71]:
def create_daily_df(prev_file, file):
    eve_data = get_tweets(prev_file, True)
    mor_data = get_tweets(file, False)
    return pd.concat([eve_data, mor_data])

In [154]:
def remove_twitter_specific(text):
    prep.set_options(
          prep.OPT.URL
        , prep.OPT.MENTION
    # Dont' remove hashtags instead convert them to normal words
    #    , prep.OPT.HASHTAG
    # Removes for example RT and similiar twitter reserved words    
        , prep.OPT.RESERVED
        , prep.OPT.EMOJI
        , prep.OPT.SMILEY)
    return prep.clean(text)

def remove_punctuation(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = text.replace('“', '').replace('”', '').replace('…', '')
    return text.lower()

def stemming(text, stemmer):
    text_list =  [stemmer.stem(word) for word in text.split(' ')]
    #text = [word for word in text if word != ' ']
    return " ".join(text_list)
    
def clean_tweet(text, stemmer):
    text = remove_twitter_specific(text)
    text = remove_punctuation(text)
    #text = stemming(text, stemmer)
    return text.strip()
    

def create_cleaned_csv(df, comp, stemmer):
    df = df['Content']
    df = df.apply(lambda x: clean_tweet(x, stemmer))
    date = file.split('.')[0][len(file)-14:]
    df.to_csv('data/tweets/unstemmed/%s_%s.csv' % (comp, date), index=False)

In [156]:
prev_comp = ''
prev_file = ''
stemmer = nltk.PorterStemmer()
for counter, file in enumerate(glob.glob('data/tweets/temp/*.csv')):
    comp = file.split('\\')[1].split('_')[0]
    if prev_comp == comp:
        df = create_daily_df(prev_file, file)
        create_cleaned_csv(df, comp, stemmer)
    prev_comp = comp
    prev_file = file